In [97]:
import os
import pandas as pd
import math
import copy
import numpy as np
from tqdm import tqdm

In [98]:
cwd = os.getcwd()
parent_directory = os.path.abspath(os.path.join(cwd, os.pardir))

path_to_samples = parent_directory + '/example_dataset_derivatives/sub-ab01/ses-second/samples.hdf5'
samples = pd.read_hdf(path_or_buf=path_to_samples)

In [125]:

def fixations_saccades_detection(samples, sac_max_vel=1500, fix_max_amp=1.5, sac_time_thresh=0.002,
                                 drop_fix_from_blink=True, sfreq=1000,
                                 screen_size=38, screen_resolution=1920, screen_distance=60,
                                 out_fname='default_name.tsv', out_folder='Remodnav_detection/'):

    """
    Run events detection using Remodnav mthod based on Nystrom and ... algorithm (2010)


    Only on one eye so far

    Only runs for part of the samples (??

    Must review the parameters used

    Screen_distance and screen_size must have matching units
    
    """

    # sac_max_vel=1500
    # fix_max_amp=1.5
    # sac_time_thresh=0.002                
    # drop_fix_from_blink=True
    # sfreq=1000
    # screen_size=38
    # screen_resolution=1920
    # screen_distance=60
    # out_fname='default_name.tsv'
    # out_folder='Remodnav_detection/'

    samples = samples[:11281]
    times = np.arange(stop=len(samples) / sfreq, step=1/sfreq)
    gazex_data = samples['LX']
    gazey_data = samples['RX']
    pupil_data = samples['LPupil']


    # If not pre run data, run
    print('\nRunning saccades and fixations detection')

    # Define data to save to excel file needed to run the saccades detection program Remodnav
    eye_data = {'x': gazex_data, 'y': gazey_data}
    df = pd.DataFrame(eye_data)

    # Remodnav parameters
    fname = f'eye_data.csv'  # eye data file to use as input for Remodnav 
    px2deg = math.degrees(math.atan2(.5 * screen_size, screen_distance)) / (.5 * screen_resolution)

    # Save csv file
    df.to_csv(fname, sep='\t', header=False, index=False)

    # Run Remodnav not considering pursuit class and min fixations 50 ms
    command = f'remodnav {fname} {out_fname} {px2deg} {sfreq} --min-pursuit-duration {2} ' \
                f'--max-pso-duration {0.0} --min-fixation-duration {0.05} --max-vel {1000}'
    failed = os.system(command)

    # Raise error if events detection with Remodnav failed
    if failed:
        raise ValueError('Remodnav detection failed')

    # Read results file with detections
    sac_fix = pd.read_csv(out_fname, sep='\t')

    # Move eye data, detections file and image to subject results directory
    os.makedirs(out_folder, exist_ok=True)
    # Move et data file
    os.replace(fname, out_folder + fname)
    # Move results file
    os.replace(out_fname, out_folder + out_fname)
    # Move results image
    out_fname = out_fname.replace('tsv', 'png')
    os.replace(out_fname, out_folder + out_fname)

    # Get saccades and fixations
    saccades_all = copy.copy(sac_fix.loc[(sac_fix['label'] == 'SACC') | (sac_fix['label'] == 'ISAC')])
    fixations_all = copy.copy(sac_fix.loc[sac_fix['label'] == 'FIXA'])

    # Drop saccades and fixations based on conditions
    print(f'Dropping saccades with average vel > {sac_max_vel}, and fixations with amplitude > {fix_max_amp}')

    fixations = copy.copy(fixations_all[(fixations_all['amp'] <= fix_max_amp)])
    saccades = copy.copy(saccades_all[saccades_all['peak_vel'] <= sac_max_vel])

    print(f'Kept {len(fixations)} out of {len(fixations_all)} fixations')
    print(f'Kept {len(saccades)} out of {len(saccades_all)} saccades')

    # Variables to save fixations features
    mean_x = []
    mean_y = []
    pupil_size = []
    prev_sac = []
    next_sac = []

    # Identify neighbour saccades to each fixation (considering sac_time_thresh)
    print('Finding previous and next saccades')

    for fix_idx, fixation in tqdm(fixations.iterrows(), total=len(fixations)):

        fix_time = fixation['onset']
        fix_dur = fixation['duration']

        # Previous and next saccades
        try:
            sac0 = saccades.loc[(saccades['onset'] + saccades['duration'] > fix_time - sac_time_thresh) & (
                        saccades['onset'] + saccades['duration'] < fix_time + sac_time_thresh)].index.values[-1]
        except:
            sac0 = None
        prev_sac.append(sac0)

        try:
            sac1 = saccades.loc[(saccades['onset'] > fix_time + fix_dur - sac_time_thresh) & (
                        saccades['onset'] < fix_time + fix_dur + sac_time_thresh)].index.values[0]
        except:
            sac1 = None
        next_sac.append(sac1)


    # Add columns
    fixations['prev_sac'] = prev_sac
    fixations['next_sac'] = next_sac

    # Drop when no previous saccade detected in sac_time_thresh
    if drop_fix_from_blink:
        fixations.dropna(subset=['prev_sac'], inplace=True)
        print(f'\nKept {len(fixations)} fixations with previous saccade')


    # Fixations features
    print('Computing average pupil size, and x and y position')

    for fix_idx, fixation in tqdm(fixations.iterrows(), total=len(fixations)):

        fix_time = fixation['onset']
        fix_dur = fixation['duration']

        # Average pupil size, x and y position
        fix_time_idx = np.where(np.logical_and(times > fix_time, times < fix_time + fix_dur))[0]

        pupil_data_fix = pupil_data[fix_time_idx]
        gazex_data_fix = gazex_data[fix_time_idx]
        gazey_data_fix = gazey_data[fix_time_idx]

        pupil_size.append(np.nanmean(pupil_data_fix))
        mean_x.append(np.nanmean(gazex_data_fix))
        mean_y.append(np.nanmean(gazey_data_fix))

    fixations['mean_x'] = mean_x
    fixations['mean_y'] = mean_y
    fixations['pupil'] = pupil_size
    fixations = fixations.astype({'mean_x': float, 'mean_y': float, 'pupil': float, 'prev_sac': 'Int64', 'next_sac': 'Int64'})

    return fixations, saccades, times

In [126]:
fixations, saccades, time = fixations_saccades_detection(samples=samples)


Running saccades and fixations detection
Dropping saccades with average vel > 1500, and fixations with amplitude > 1.5
Kept 21 out of 24 fixations
Kept 24 out of 24 saccades
Finding previous and next saccades


100%|██████████| 21/21 [00:00<00:00, 1504.77it/s]



Kept 18 fixations with previous saccade
Computing average pupil size, and x and y position


100%|██████████| 18/18 [00:00<00:00, 1389.20it/s]


In [130]:
fixations.head()

,onset,duration,label,start_x,start_y,end_x,end_y,amp,peak_vel,med_vel,avg_vel,prev_sac,next_sac,mean_x,mean_y,pupil
4,0.848,0.162,FIXA,961.49071,933.54803,954.65891,911.87010,0.416,106.791,8.857,13.734,3,5,941.242857,909.151553,627.180124
6,1.026,1.691,FIXA,972.32388,946.02999,972.64790,931.80057,0.261,25.959,4.435,4.916,5,7,973.259645,946.606450,531.368047
8,2.728,0.275,FIXA,961.54631,925.15241,959.81504,927.06369,0.047,26.445,5.864,6.569,7,9,967.109489,930.754745,506.722628
10,3.016,0.839,FIXA,978.85759,949.42242,967.27320,947.05524,0.216,36.098,5.021,5.593,9,11,971.318019,947.872076,518.249403
12,3.873,0.130,FIXA,863.24931,845.73220,844.59412,846.51066,0.342,138.744,13.213,21.339,11,13,880.042636,859.696899,524.984496
